In [1]:
import xarray as xr
import xesmf as xe
import pandas as pd
import datetime
import os

In [26]:
first_date = '2021-01-01'
last_date  = '2022-12-31'

lonmin,lonmax = -90,-69
latmin,latmax = -40,-15

variables = [
    'surf_el',
    'water_temp',
    'salinity',
    'water_u',
    'water_v']



In [29]:
def get_hycom_filename(ftype):
    if ftype=='hindcast':
        url  = 'https://<Lucas.Glasner:y4vkrp7lqcv>@tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0'
    if ftype=='reanalysis':
        url  = 'https://<Lucas.Glasner:y4vkrp7lqcv>@tds.hycom.org/thredds/dodsC/GLBv0.08/expt_53.X/data'
    return url


def get_hycom_hindcast(first_date, last_date, lonmin, lonmax, latmin, latmax, variables):
    url  = get_hycom_filename('hindcast')
    data = xr.open_dataset(url, decode_times=False)
    data = data[variables]
    data = data.sel(lat=slice(latmin,latmax), lon=slice(lonmin, lonmax))
    units,reference_date =  data.time.attrs['units'].split('since')

    time = [pd.Timedelta(hours=t)+pd.to_datetime(reference_date) for t in data.time.values]
    data.coords['time'] = ('time',time,data.time.attrs)
    data = data.sel(time=slice(first_date, last_date))
    return data
    
def get_hycom_reanalysis(year, lonmin, lonmax, latmin, latmax, variables):
    url  = get_hycom_filename('reanalysis')+'/{}'.format(year)
    data = xr.open_dataset(url, decode_times=False)
    data = data[variables]
    data = data.sel(lat=slice(latmin,latmax), lon=slice(lonmin,lonmax))
    units,reference_date =  data.time.attrs['units'].split('since')
    time = [pd.Timedelta(hours=t)+pd.to_datetime(reference_date) for t in data.time.values]
    data.coords['time'] = ('time',time,data.time.attrs)
    return data

In [30]:
data = get_hycom_reanalysis(1994, lonmin, lonmax, latmin, latmax, variables)